SISTEMA EXPERTO

Valentin Moreno Vasquez

instalacion de librerias

In [ ]:
!pip uninstall openai pydantic -y
!pip install openai==1.61.1 pydantic==1.10.13
import os
os._exit(0)

Found existing installation: openai 1.61.1
Uninstalling openai-1.61.1:
  Successfully uninstalled openai-1.61.1
Found existing installation: pydantic 1.10.13
Uninstalling pydantic-1.10.13:
  Successfully uninstalled pydantic-1.10.13
  Using cached openai-1.61.1-py3-none-any.whl.metadata (27 kB)
  Using cached pydantic-1.10.13-py3-none-any.whl.metadata (149 kB)
Using cached openai-1.61.1-py3-none-any.whl (463 kB)
Using cached pydantic-1.10.13-py3-none-any.whl (158 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.23.0 requires pydantic<3.0.0,>=2.0, but you have pydantic 1.10.13 which is incompatible.
gradio 5.50.0 requires pydantic<=2.12.3,>=2.0, but you have pydantic 1.10.13 which is incompatible.
albumentations 2.0.8 requires pydantic>=2.9.2, but you have pydantic 1.10.13 which is incompatible.
langsmith 0.6.8 requires pydantic<3,>=2, but you ha

In [ ]:
import openai
print(openai.__version__)

1.61.1


In [ ]:
# Importación de librerías necesarias para la funcionalidad del asistente
import os         # Manejo de archivos y directorios
import requests   # Realización de peticiones HTTP (aunque no se usa en este código)
import tempfile   # Creación de archivos y directorios temporales
import shutil     # Copia y eliminación de archivos y carpetas
import glob       # Búsqueda de archivos en directorios
import time       # Manejo de pausas y tiempos de espera
from google.colab import drive  # Librería para montar Google Drive en Colab

# Montar Google Drive en el entorno de Google Colab
# Esto permite acceder a los archivos almacenados en la nube
drive.mount('/content/drive')

# Definir la carpeta donde se encuentran los documentos de referencia en Google Drive
# Estos documentos contienen información sobre las leyes de tránsito colombianas
REFERENCE_FOLDER = "/content/drive/MyDrive/openiaclase/"

def get_files_from_folder(folder_path):
    """Obtiene una lista con las rutas de todos los archivos dentro de la carpeta especificada."""
    return glob.glob(os.path.join(folder_path, "*"))  # Busca todos los archivos en la ruta dada


# Definición de la clase InsuranceAssistant, que representa el asistente virtual de tránsito
class InsuranceAssistant:
    def __init__(self, api_key_path, model="gpt-4o-mini",
                 instructions="""Eres un agente de tránsito colombiano experto en las leyes de tránsito de este país.
                 A partir del documento de las leyes vigentes, debes ser capaz de extraer información efectiva de lo que se pregunte acerca de esta ley y que hacer en los casos correspondientes.
                 Sé lo más preciso posible con el fin de que los agentes puedan responder preguntas exactas o, en el caso de una persona, pueda tener certeza sobre lo que está preguntando.
                 Da toda la información en el idioma en el que se te pregunte."""):

        # Leer la clave API desde el archivo proporcionado en api_key_path
        # Esto permite la autenticación con OpenAI para poder usar su API
        with open(api_key_path, "r") as file:
            openai.api_key = file.read().strip()  # Elimina espacios en blanco al inicio y al final

        self.instructions = instructions  # Instrucciones personalizadas para el asistente
        self.model = model  # Modelo de OpenAI que se usará para generar respuestas

        # Crear un Vector Store en OpenAI para almacenar documentos relevantes
        # Un Vector Store es una base de datos que permite hacer búsquedas semánticas
        self.vector_store = openai.beta.vector_stores.create(name="Car Insurance Files")

        # Crear el asistente con el modelo GPT-4o-mini y configurar su comportamiento
        self.assistant = openai.beta.assistants.create(
            name="Insurance Analysis Assistant",  # Nombre del asistente
            instructions=self.instructions,  # Instrucciones de comportamiento
            model=self.model,  # Modelo a utilizar para responder preguntas
            tools=[{"type": "file_search"}],  # Herramienta que le permite buscar en documentos
            tool_resources={"file_search": {"vector_store_ids": [self.vector_store.id]}}  # Vincula el asistente con el Vector Store
        )

        # Cargar automáticamente los documentos desde la carpeta de referencia en Google Drive
        self.add_documents(get_files_from_folder(REFERENCE_FOLDER))

    def add_documents(self, file_paths):
        """Añade documentos al Vector Store para que el asistente pueda usarlos en sus respuestas."""
        for path in file_paths:  # Itera sobre cada archivo en la lista de archivos
            uploaded_file = openai.files.create(
                file=open(path, "rb"),  # Abre el archivo en modo lectura binaria
                purpose="assistants"  # Especifica que el archivo será usado por un asistente
            )

            # Agrega el archivo al Vector Store para permitir búsquedas en él
            openai.beta.vector_stores.files.create(
                vector_store_id=self.vector_store.id,  # ID del Vector Store al que se añade el archivo
                file_id=uploaded_file.id  # ID del archivo subido
            )

    def list_documents(self):
        """Lista los documentos almacenados en el Vector Store del asistente."""
        files = openai.beta.vector_stores.files.list(vector_store_id=self.vector_store.id)  # Obtiene los archivos almacenados
        # Retorna una lista con el ID y el nombre de cada archivo
        return [{"id": f.id, "name": getattr(f, "filename", "Desconocido")} for f in files.data]

    def remove_document(self, file_id):
        """Elimina un documento específico del Vector Store usando su ID."""
        try:
            openai.beta.vector_stores.files.delete(
                vector_store_id=self.vector_store.id,  # ID del Vector Store del asistente
                file_id=file_id  # ID del archivo que se quiere eliminar
            )
            return f"Archivo {file_id} eliminado correctamente."  # Mensaje de éxito
        except openai.OpenAIError as e:
            return f"Error al eliminar el archivo: {e}"  # Mensaje de error si algo falla

    def ask(self, question):
        """Realiza una consulta al asistente y obtiene una respuesta basada en los documentos almacenados."""

        # Crea un nuevo hilo de conversación (thread) en OpenAI
        thread = openai.beta.threads.create()

        # Agrega la pregunta del usuario al hilo de conversación
        openai.beta.threads.messages.create(
            thread_id=thread.id,  # ID del hilo donde se almacena la pregunta
            role="user",  # Indica que el mensaje es del usuario
            content=question  # Contenido de la pregunta
        )

        # Inicia la ejecución del asistente en el hilo de conversación
        run = openai.beta.threads.runs.create(
            thread_id=thread.id,  # ID del hilo donde se ejecutará la consulta
            assistant_id=self.assistant.id,  # ID del asistente que responderá la consulta
        )

        # Espera hasta que el asistente haya generado una respuesta
        while run.status not in ["completed", "failed"]:  # Mientras el estado no sea "completado" o "fallido"
            time.sleep(2)  # Pausa de 2 segundos para evitar hacer demasiadas peticiones seguidas
            run = openai.beta.threads.runs.retrieve(
                thread_id=thread.id,  # ID del hilo de la consulta
                run_id=run.id  # ID de la ejecución en curso
            )

        # Si la ejecución falla, retorna un mensaje de error
        if run.status == "failed":
            return "Error en la generación de respuesta."

        # Obtiene los mensajes generados en el hilo de conversación
        messages = openai.beta.threads.messages.list(thread_id=thread.id)

        # Busca el mensaje generado por el asistente
        for msg in messages.data:
            if msg.role == "assistant":  # Si el mensaje es del asistente
                return msg.content[0].text.value  # Devuelve el contenido de la respuesta

        return "No se encontró respuesta válida."  # Mensaje en caso de que no haya respuesta del asistente

Mounted at /content/drive


Consulta

In [ ]:
# Inicializar tu asistente desde Colab
asistente = InsuranceAssistant("/content/drive/MyDrive/openiaclase/api_key_openai.txt")

# Realiza una pregunta de prueba:
respuesta = asistente.ask("Que se debe hace en caso de que una persona me choque en carretera.")
print(respuesta)

En caso de que una persona te choque en carretera, debes seguir estos pasos según la legislación de tránsito colombiana:

1. **Detenerse y verificar la situación**: Después de un choque, detén tu vehículo de inmediato y asegúrate de que todos los involucrados estén a salvo. Evalúa si hay lesiones personales o daños considerables.

2. **Llamar a las autoridades**: Si hay heridos o daños significativos, llama a la policía de tránsito o a los servicios de emergencia. Ellos son los encargados de gestionar la situación y crear el informe correspondiente.

3. **No mover los vehículos inicialmente**: A menos que sea necesario para evitar más accidentes, no debes mover los vehículos hasta que lleguen las autoridades. Esto es importante para la elaboración del informe de accidente.

4. **Intercambiar información**: Asegúrate de intercambiar información con el otro conductor. Esto incluye tu nombre, número de documento de identidad, número de licencia de conducción, datos del vehículo (marca, mo